In [1]:
import requests
import json 
import random 

In [2]:
presidential_national_vote_url = "https://www.politico.com/election-data/results__2024-11-05__collections__2024-11-05-collection-president__summaries/data.json"
presidential_county_base_url_prefix = "https://www.politico.com/election-data/results__2024-11-05__contests__ap:" 
presidential_national_vote_url_suffix = ":0__counties/data.json"

In [3]:
with open("us_states.json", "r") as infile:
    national_feature_map = json.load(infile)
with open("candidate_ids.json", "r") as infile:
    candidates = json.load(infile)
with open("county_feature_list.json", "r") as infile:
    county_feature_map = json.load(infile)



In [4]:
v_request = requests.get(presidential_national_vote_url)
v_cont = v_request.content
v_dict = json.loads(v_cont)

In [5]:
for contest in v_dict["contests"]:
    tmp_id = contest["id"][3:5]
    harris = 0 
    trump = 0 
    third_party = 0 
    progress = contest['progress']['pct']
    for result in contest['results']:
        if candidates[result['id']] == 'Donald Trump': 
            trump += result['votes'] 
        elif candidates[result['id']] == 'Kamala Harris': 
            harris += result['votes']
        else: 
            third_party += result['votes']
    if harris+trump+third_party != 0:
        harris = harris/(harris+trump+third_party)
        trump = trump/(harris+trump+third_party)
        third_party = third_party/(harris+trump+third_party)
    for i in national_feature_map['features']:
        if i['id'] == tmp_id:
            tmp_property_dict = i['properties']
            tmp_property_dict["harris"] = harris
            tmp_property_dict["trump"] = trump
            tmp_property_dict["third_party"] = third_party
            tmp_property_dict["progress"] = progress
            tmp_property_dict["margin"] = harris - trump
            tmp_property_dict["margin"] = random.randint(-15,15)
            i["properties"] = tmp_property_dict
with open("us_states.js", "w") as outfile:
    outfile.write(f"var statesData = {json.dumps(national_feature_map)}")

In [6]:
for feature in national_feature_map['features']:
    state_request = requests.get(f"{presidential_county_base_url_prefix}{feature['id']}{presidential_national_vote_url_suffix}")
    state_content = state_request.content
    state_json = json.loads(state_content)
    if contest["id"] == "02": 
        print(state_json)
        break 
    for contest in state_json["counties"]:
        
        tmp_id = str(int(contest["id"]))
            
        #print(tmp_id)
        harris = 0 
        trump = 0 
        third_party = 0 
        progress = contest['progress']['pct']
        for result in contest['results']:
            if candidates[result['id']] == 'Donald Trump': 
                trump += result['votes'] 
            elif candidates[result['id']] == 'Kamala Harris': 
                harris += result['votes']
            else: 
                third_party += result['votes']
        if harris+trump+third_party != 0:
            harris = harris/(harris+trump+third_party)
            trump = trump/(harris+trump+third_party)
            third_party = third_party/(harris+trump+third_party)
        for i in county_feature_map['features']:
            if i['id'] == tmp_id:
                tmp_property_dict = i['properties']
                tmp_property_dict["state"] = feature["properties"]["name"]
                tmp_property_dict["harris"] = harris
                tmp_property_dict["trump"] = trump
                tmp_property_dict["third_party"] = third_party
                tmp_property_dict["progress"] = progress
                tmp_property_dict["margin"] = harris - trump
                # tmp_property_dict["margin"] = random.randint(-15,15)
                i["properties"] = tmp_property_dict

with open("us_counties.js", "w") as outfile:
    outfile.write(f"var countyData = {json.dumps(county_feature_map)}")

KeyboardInterrupt: 